In [1]:
import pybaseball
from pybaseball import statcast as sc
import pandas as pd
import numpy as np

In [2]:
#sc?

Enable the cache because we are querying a lot of data, and making sure we can see every column of a large dataframe when we call df.head()

In [3]:
pybaseball.cache.enable()
pd.set_option('display.max_columns', None)

Querying all statcast data from the range of games including all astros home games in 2017 (also all other games in that span)

In [7]:
data = sc(start_dt = '2017-4-3', end_dt = '2017-11-01')

This is a large query, it may take a moment to complete


100%|██████████| 213/213 [00:14<00:00, 14.57it/s]


In [8]:
data.shape

(731564, 92)

Subset so we only have Astros at-bats left

In [9]:
data = data[(data['home_team'] == 'HOU') & (data['inning_topbot'] == 'Bot')]

In [10]:
data.shape

(12771, 92)

In [14]:
data.columns

Index(['pitch_type', 'game_date', 'release_speed', 'release_pos_x',
       'release_pos_z', 'player_name', 'batter', 'pitcher', 'events',
       'description', 'spin_dir', 'spin_rate_deprecated',
       'break_angle_deprecated', 'break_length_deprecated', 'zone', 'des',
       'game_type', 'stand', 'p_throws', 'home_team', 'away_team', 'type',
       'hit_location', 'bb_type', 'balls', 'strikes', 'game_year', 'pfx_x',
       'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b',
       'outs_when_up', 'inning', 'inning_topbot', 'hc_x', 'hc_y',
       'tfs_deprecated', 'tfs_zulu_deprecated', 'fielder_2', 'umpire', 'sv_id',
       'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot',
       'hit_distance_sc', 'launch_speed', 'launch_angle', 'effective_speed',
       'release_spin_rate', 'release_extension', 'game_pk', 'pitcher.1',
       'fielder_2.1', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6',
       'fielder_7', 'fielder_8', 'fielder_9', 'release_pos_y',
       'estima

Next we take the first game of the Astros season, their home opener which has game_pk 490111

In [7]:
# game_pk_490111 = data[data['game_pk'] == 490111]

In [8]:
# game_pk_490111.shape

(104, 92)

Just looking at Astros at-bats

sorting from the beginning to the end of the game by at_bat_number and pitch number, so the order in which they are presented is the order in which they happened

In [15]:
# game_pk_490111 = game_pk_490111.sort_values(['at_bat_number', 'pitch_number'],ascending = [True, True])
data = data.sort_values(['game_date', 'at_bat_number', 'pitch_number'],ascending = [True, True, True])

We want player_name to be the name of the batter, not the pitcher. To fix this, I pulled every player_id from every Astros at-bat from data, and used playerid_reverse_lookup to get their names. I formatted them to match the player_name formatting, and added the column to game_pk_490111 as a test

In [17]:
from pybaseball import playerid_reverse_lookup

player_ids = data['batter'].tolist()

# find the names of the players in player_ids, along with their ids from other data sources
names = playerid_reverse_lookup(player_ids, key_type='mlbam')[['name_last','name_first','key_mlbam']]
names['player_name'] = (names['name_last'].str.capitalize() + ', ' + names['name_first'].str.capitalize())
names = names.drop(['name_last', 'name_first'], axis = 1)

Gathering player lookup table. This may take a moment.


In [18]:
names.head()

,key_mlbam,player_name
0,514888,"Altuve, Jose"
1,493114,"Aoki, Nori"
2,136860,"Beltran, Carlos"
3,608324,"Bregman, Alex"
4,518542,"Centeno, Juan"


In [19]:
names = names.set_index('key_mlbam').T.to_dict('list')

In [20]:
for i in names:
    names[i] = str(names[i])[2:-2]

In [21]:
# game_pk_490111['player_name'] = game_pk_490111['batter'].map(names)
data['player_name'] = data['batter'].map(names)

In [17]:
#if you'd like it as an excel sheet, uncomment this. it should appear in the same folder as this file
#game_pk_490111.to_excel('game_pk_490111_python.xlsx')

In [24]:
# game_pk_490111
data.to_csv('astros_2017_home_games.csv', index=False)

And that's it! this is the exact same as the baseball savant data from the baseballr package that I cleaned, but this can be automated.

now, all you have to do is download the MLBAM data for every astros game, this data for every astros game, clean it like this, and then you can put them side by side. then, match up the playID from the MLBAM and sign stealing data to put it all together.